In [17]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.layers import Conv2D
from keras.layers import Add
from keras.models import Model


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_path_1 = "/content/drive/My Drive/IMDDataSample1deg_1981_2021.npy"
file_path_2 = "/content/drive/My Drive/BiasDataSample1deg_1981_2021.npy"
data_1 = np.load(file_path_1)
data_2 = np.load(file_path_2)

In [4]:
data_1.shape

(2132, 36, 36)

In [5]:
data_2.shape

(2132, 36, 36)

In [6]:
data_1

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [7]:
data_2

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [8]:
high_res_images = data_1[:1716, :, :]
low_res_images=data_2[:1716,:,:]

In [9]:
scaler = StandardScaler()
low_res_images = scaler.fit_transform(low_res_images.reshape(-1, 1)).reshape(low_res_images.shape)
high_res_images = scaler.transform(high_res_images.reshape(-1, 1)).reshape(high_res_images.shape)


In [10]:
X_train, X_temp, y_train, y_temp = train_test_split(low_res_images, high_res_images, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [15]:
def srdrn_block(x):
    # First convolutional layer
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)

    # Recursive layers (adjust the number of recursive blocks as needed)
    for _ in range(4):  # Experiment with the number of recursive blocks
        residual = x
        x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
        x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
        x = Add()([residual, x])  # Residual connection

    # Last convolutional layer
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)

    return x

# Input layer
input_layer = keras.layers.Input(shape=(36, 36, 1))

# Initial convolutional layer
x = Conv2D(64, (3, 3), padding='same', activation='relu')(input_layer)

# SRDRN blocks
for _ in range(8):  # Experiment with the number of SRDRN blocks
    x = srdrn_block(x)

# Output layer
output_layer = Conv2D(1, (3, 3), padding='same', activation='linear')(x)

In [18]:
# Create the SRDRN model
srdrn_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
srdrn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Define early stopping and model checkpointing callbacks
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = keras.callbacks.ModelCheckpoint('srdrn_model.h5', save_best_only=True)


In [19]:
history = srdrn_model.fit(X_train, y_train, validation_data=(X_val, y_val),
                          epochs=100, batch_size=32, callbacks=[early_stopping, model_checkpoint])

Epoch 1/100
38/38 [==============================] - 49s 255ms/step - loss: 10.7236 - mean_absolute_error: 1.2240 - val_loss: 1.6992 - val_mean_absolute_error: 0.8454
Epoch 2/100
38/38 [==============================] - 8s 214ms/step - loss: 1.6016 - mean_absolute_error: 0.8762 - val_loss: 1.4151 - val_mean_absolute_error: 1.0088
Epoch 3/100
38/38 [==============================] - 8s 209ms/step - loss: 1.4378 - mean_absolute_error: 0.8157 - val_loss: 1.7267 - val_mean_absolute_error: 0.8308
Epoch 4/100
38/38 [==============================] - 8s 209ms/step - loss: 1.6187 - mean_absolute_error: 0.8929 - val_loss: 2.3333 - val_mean_absolute_error: 0.8187
Epoch 5/100
38/38 [==============================] - 8s 218ms/step - loss: 1.5972 - mean_absolute_error: 0.9375 - val_loss: 0.9375 - val_mean_absolute_error: 0.7884
Epoch 6/100
38/38 [==============================] - 8s 212ms/step - loss: 1.4909 - mean_absolute_error: 0.7731 - val_loss: 3.3985 - val_mean_absolute_error: 1.3937
Epoch 7/

In [20]:
best_model = keras.models.load_model('srdrn_model.h5')

In [21]:
test_loss, test_mae = best_model.evaluate(X_test, y_test)

9/9 [==============================] - 1s 74ms/step - loss: 0.0571 - mean_absolute_error: 0.1023


In [22]:
bias_corrected_data = best_model.predict(low_res_images)

54/54 [==============================] - 5s 71ms/step


In [23]:
bias_corrected_data = bias_corrected_data.reshape(-1, 36, 36)

In [24]:
GCM_data_real=high_res_images.reshape(-1,36,36)

In [25]:
differences=bias_corrected_data-GCM_data_real

In [26]:
mean_differences=np.mean(differences,axis=(1,2))
global_mean_difference=np.mean(differences)
print(global_mean_difference)

-0.008110182
